In [21]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import json
import pickle

In [22]:
# Import and download stopwords from NLTK.
from nltk import download
download('stopwords')  # Download stopwords list.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from google.colab import drive
import os
import sys

#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

sys.path.append('/content/gdrive/My Drive/PLP/')
print("All the system paths:")
sys.path

os.chdir('/content/drive/My Drive/PLP-Project')
print("Current working directory:")
os.getcwd()

Mounted at /content/drive
All the system paths:
Current working directory:


'/content/drive/My Drive/PLP-Project'

In [24]:
#df = pd.read_json ('dataset/df_ca.json')
#with open('/content/drive/MyDrive/PLP-Project/dataset/df_CA.json') as json_file:   

with open('dataset/df_CA.json') as json_file:     
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

df = pd.DataFrame(data)

In [25]:
df.head()

,education,gPlusUserId,address,closed,gps,hours,name,phone,price,categories,gPlusPlaceId,rating,reviewText,reviewCount,userCount,jobs,loc
0,"[[[], [], [], [], [], 6], [[""The University of...",1.002152e+20,"[546 University Ave, Palo Alto, CA 94301]",False,"[37.448889, -122.158481]","[[Monday, [[""11:30 am--2:30 pm""],[""5:00--9:00 ...",Tamarine,(650) 325-8500,$,"[Restaurant, Asian Restaurant, Southeast Asian...",1.016694e+20,5,A bit expensive but lunch specials are reasona...,71,3,"[[Google, Engineer, [[],[1,1,2011],1], , ]]",California
1,"[[[], [], [], [], [], 6], [[""University of Mis...",1.019804e+20,"[990 Valencia St, San Francisco, CA 94110]",False,"[37.757165, -122.421287]","[[Monday, [[""7:00 am--6:00 pm""]], 1], [Tuesday...",Blue Fig,(415) 875-9622,$$$,"[Restaurant, Cafe]",1.169607e+20,4,A casual friendly spot that where nothing shor...,13,28,"[[Retailigence, Director of Business Developme...",California
2,"[[[], [], [], [], [], 6], [[""University of Mis...",1.019804e+20,"[452 University Ave, Palo Alto, CA 94301]",False,"[37.447589, -122.159776]","[[Monday, [[""11:00 am--10:00 pm""]], 1], [Tuesd...",Umami Burger,(650) 321-8626,None,"[Restaurant, Hamburger Restaurant, Salad Shop]",1.134952e+20,4,Delicious burgers as you'd expect from other U...,24,28,"[[Retailigence, Director of Business Developme...",California
3,"[[[], [], [], [], [], 6], [[""University of Mis...",1.019804e+20,"[655 Divisadero St, San Francisco, CA 94117]",False,"[37.775721, -122.438054]","[[Monday, [[""Closed""]], 1], [Tuesday, [[""5:00-...",BAR CRUDO,(415) 409-0679,$$,"[Restaurant, Bar, Seafood Restaurant]",1.141552e+20,5,What I'd normally consider foofoo based on the...,40,28,"[[Retailigence, Director of Business Developme...",California
4,"[[[], [], [], [], [], 6], [[""University of Mis...",1.019804e+20,"[620 Treat Ave, San Francisco, CA 94110]",False,"[37.760144, -122.413819]","[[Monday, [[""11:00 am--12:00 am""]], 1], [Tuesd...",Southern Pacific Brewing,(415) 341-0152,None,"[Bar, Brewery, Restaurant]",1.177716e+20,5,A large almost open air brewery that's able to...,38,28,"[[Retailigence, Director of Business Developme...",California


In [26]:
df.shape

(5236, 17)

Cleaning & Preparing Data

In [27]:
df1 =  df[['reviewText','rating']]

In [28]:
df1.head

<bound method NDFrame.head of                                              reviewText  rating
0     A bit expensive but lunch specials are reasona...       5
1     A casual friendly spot that where nothing shor...       4
2     Delicious burgers as you'd expect from other U...       4
3     What I'd normally consider foofoo based on the...       5
4     A large almost open air brewery that's able to...       5
...                                                 ...     ...
5231  Food was good, but like previous post said I o...       4
5232  Pizza is out of this world. I work at a Temecu...       5
5233  Delicious food for everyone! I love bringing m...       5
5234  Denica's is a staple for my family and many of...       4
5235  The food was just like any other McDonalds. Ju...       3

[5236 rows x 2 columns]>

In [29]:
df_neg =  df1.loc[df1.rating<3,['reviewText','rating']]
df_neg

,reviewText,rating
20,This place doesn't respect reservations. It's ...,1
32,"Long story short: the food was good, but the s...",2
45,We went here for the first time a couple weeks...,2
47,The success of any mediocre sushi joint relies...,1
52,Don't waste your time or your money! This rest...,1
...,...,...
5176,Last time I tried using my debit card they wan...,1
5198,Food was okay but left hungry enough for anoth...,2
5207,"Bad Service, bad food in all elephant bars",1
5213,"The portions are large, but the food is greasy...",2


In [30]:
# Make negative review number balance to positive review number  Positive:Negative as 4702:4806
df2 =  df1.append([df_neg]*8,ignore_index=True)
df2

,reviewText,rating
0,A bit expensive but lunch specials are reasona...,5
1,A casual friendly spot that where nothing shor...,4
2,Delicious burgers as you'd expect from other U...,4
3,What I'd normally consider foofoo based on the...,5
4,A large almost open air brewery that's able to...,5
...,...,...
9503,Last time I tried using my debit card they wan...,1
9504,Food was okay but left hungry enough for anoth...,2
9505,"Bad Service, bad food in all elephant bars",1
9506,"The portions are large, but the food is greasy...",2


In [31]:
df2.loc[df2.rating<3]

,reviewText,rating
20,This place doesn't respect reservations. It's ...,1
32,"Long story short: the food was good, but the s...",2
45,We went here for the first time a couple weeks...,2
47,The success of any mediocre sushi joint relies...,1
52,Don't waste your time or your money! This rest...,1
...,...,...
9503,Last time I tried using my debit card they wan...,1
9504,Food was okay but left hungry enough for anoth...,2
9505,"Bad Service, bad food in all elephant bars",1
9506,"The portions are large, but the food is greasy...",2


In [32]:
df2.loc[df2.rating>=3]

,reviewText,rating
0,A bit expensive but lunch specials are reasona...,5
1,A casual friendly spot that where nothing shor...,4
2,Delicious burgers as you'd expect from other U...,4
3,What I'd normally consider foofoo based on the...,5
4,A large almost open air brewery that's able to...,5
...,...,...
5231,"Food was good, but like previous post said I o...",4
5232,Pizza is out of this world. I work at a Temecu...,5
5233,Delicious food for everyone! I love bringing m...,5
5234,Denica's is a staple for my family and many of...,4


In [33]:
y = df2['rating']
X = df2.drop(columns=['rating'])
X.shape

(9508, 1)

In [34]:
y.isnull().sum()

0

In [35]:
for i in range(0,len(y)):
    if (y[i]>=3):
        y[i] = "Positive"
    else:
        y[i] = "Negative"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
print(y[y=='Negative'])

20      Negative
32      Negative
45      Negative
47      Negative
52      Negative
          ...   
9503    Negative
9504    Negative
9505    Negative
9506    Negative
9507    Negative
Name: rating, Length: 4806, dtype: object


In [37]:
print(y[y=='Positive'])

0       Positive
1       Positive
2       Positive
3       Positive
4       Positive
          ...   
5231    Positive
5232    Positive
5233    Positive
5234    Positive
5235    Positive
Name: rating, Length: 4702, dtype: object


In [38]:
y_o = df1['rating']
X_o = df1.drop(columns=['rating'])
X_o.shape

(5236, 1)

In [39]:
for i in range(0,len(y_o)):
    if (y_o[i]>=3):
        y_o[i] = "Positive"
    else:
        y_o[i] = "Negative"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


Applying NLP Processes

In [40]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]',' ', str(X['reviewText'][i]))
    review = review.lower() #Lowering the words is very imporatant in avoiding classifying same words as different words
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #Eleminating words that do not put much value in sentences.
    review = ' '.join(review) #Reconstructing sentences
    corpus.append(review)

In [41]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus_o = []
for i in range(0, len(X_o)):
    review_o = re.sub('[^a-zA-Z]',' ', str(X_o['reviewText'][i]))
    review_o = review_o.lower() #Lowering the words is very imporatant in avoiding classifying same words as different words
    review_o = review_o.split()
    
    review_o = [ps.stem(word) for word in review_o if not word in stopwords.words('english')] #Eleminating words that do not put much value in sentences.
    review_o = ' '.join(review_o) #Reconstructing sentences
    corpus_o.append(review_o)


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=9000) #After experimenting with 7500, 5000, 2500 ...9000 worked best.
X = cv.fit_transform(corpus).toarray()

Train-Test Split

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Deciding Best Model

 MultinomialNB

In [44]:
from sklearn.naive_bayes import MultinomialNB
restaurant_review_model = MultinomialNB().fit(X_train, y_train)

In [45]:
y_pred = restaurant_review_model.predict(X_test)

In [46]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)

In [47]:
print(confusion_m)

[[1089  125]
 [  93 1070]]


In [48]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9082877576777451


In [49]:
# save the model to disk
filename = 'mnb_model_v3.pkl'
pickle.dump(restaurant_review_model, open(filename, 'wb'))
pickle.dump(cv,open('cv-model_v3.pkl','wb'))

In [50]:
# load the model from disk to score on testing data
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9082877576777451


In [51]:
# load the model from disk to score on training data
#loaded_model = pickle.load(open(filename, 'rb'))
result_train = loaded_model.score(X_train, y_train)
print(result_train)

0.9155798625718693


In [52]:
from sklearn import metrics

# classification_report
report = metrics.classification_report(y_test, y_pred,target_names=["negative", "positive"])
print(report)

              precision    recall  f1-score   support

    negative       0.92      0.90      0.91      1214
    positive       0.90      0.92      0.91      1163

    accuracy                           0.91      2377
   macro avg       0.91      0.91      0.91      2377
weighted avg       0.91      0.91      0.91      2377



In [53]:
#input_new = ['food and taste are really bad.' ] # free text input
#input_new = ['A bit expensive but lunch specials are reasonable. Excellent food and very good taste.' ] # Positive (rating as 5) review from project dataset  - predict as Positive
#input_new = ['You can\'t go wrong at this place -the whole menu rocks!' ] # Positive (rating as 5) review from project dataset  - predict as Negative
#input_new = ['This place doesn\'t respect reservations. It\'s loud, over priced and will not make you full.'] # Negative(rating as 1) review from project dataset - predict as Positive
#input_new = ['Don\t waste your time or your money! This restaurant has forgotten everything that made them what they were.  Poor to slow service. Overpriced everything. Management that is so bad I am at a loss for words.  Plenty of other options in Temecula, choose one of them. '] # Negative(rating as 1) review from project dataset - predict as Negative
#input_new_neg = ['This place used to be good but now it is disgusting and dirty stay away unless you want to get mugged or buy crack on the way out.  This is a very dangerous block.  you will never find parking anyway.  Go to Chutneys across the st.  At least it is clean and they never made me sick'] # Negative(rating as 1) review from project dataset - predict as Negative

#load and model vectorization 
input_new = ['food and taste are really bad.','A bit expensive but lunch specials are reasonable. Excellent food and very good taste.','You can\'t go wrong at this place -the whole menu rocks!','This place doesn\'t respect reservations. It\'s loud, over priced and will not make you full.','Don\t waste your time or your money! This restaurant has forgotten everything that made them what they were.  Poor to slow service. Overpriced everything. Management that is so bad I am at a loss for words.  Plenty of other options in Temecula, choose one of them. ','This place used to be good but now it is disgusting and dirty stay away unless you want to get mugged or buy crack on the way out.  This is a very dangerous block.  you will never find parking anyway.  Go to Chutneys across the st.  At least it is clean and they never made me sick']
classifier = pickle.load(open(filename, 'rb'))
cv = pickle.load(open('cv-model_v3.pkl','rb'))

vect = cv.transform(input_new).toarray()
input_prediction = classifier.predict(vect)
print(input_prediction)

['Positive' 'Positive' 'Negative' 'Positive' 'Negative' 'Negative']


In [54]:
vect1 = cv.transform(corpus).toarray()
input_prediction1 = classifier.predict(vect1)
print(input_prediction1)

['Positive' 'Positive' 'Positive' ... 'Negative' 'Positive' 'Negative']


In [55]:
confusion_m_corpus = confusion_matrix(y, input_prediction1)
print(confusion_m_corpus)

[[4338  468]
 [ 352 4350]]


In [56]:
from sklearn import metrics

# classification_report
report_c = metrics.classification_report(y, input_prediction1,target_names=["negative", "positive"])
print(report_c)

              precision    recall  f1-score   support

    negative       0.92      0.90      0.91      4806
    positive       0.90      0.93      0.91      4702

    accuracy                           0.91      9508
   macro avg       0.91      0.91      0.91      9508
weighted avg       0.91      0.91      0.91      9508



In [57]:
#load the model trained from orignal dataset total=5236 Positive:Negative as 4702:534
input_new = ['food and taste are really bad.','A bit expensive but lunch specials are reasonable. Excellent food and very good taste.','You can\'t go wrong at this place -the whole menu rocks!','This place doesn\'t respect reservations. It\'s loud, over priced and will not make you full.','Don\t waste your time or your money! This restaurant has forgotten everything that made them what they were.  Poor to slow service. Overpriced everything. Management that is so bad I am at a loss for words.  Plenty of other options in Temecula, choose one of them. ','This place used to be good but now it is disgusting and dirty stay away unless you want to get mugged or buy crack on the way out.  This is a very dangerous block.  you will never find parking anyway.  Go to Chutneys across the st.  At least it is clean and they never made me sick']

#load and model vectorization 
classifier_o = pickle.load(open('mnb_model.pkl', 'rb'))
cv_o = pickle.load(open('cv-model.pkl','rb'))

vect_o = cv_o.transform(input_new).toarray()
input_prediction_o = classifier_o.predict(vect_o)
print(input_prediction_o )

['Positive' 'Positive' 'Positive' 'Positive' 'Negative' 'Negative']


In [58]:
vect2 = cv_o.transform(corpus_o).toarray()
input_prediction2 = classifier_o.predict(vect2)

confusion_m_corpus_o = confusion_matrix(y_o, input_prediction2)
print(confusion_m_corpus_o)

[[ 324  210]
 [  90 4612]]


In [19]:
from sklearn import metrics

# classification_report
report_o = metrics.classification_report(y_o, input_prediction2,target_names=["negative", "positive"])
print(report_o)

              precision    recall  f1-score   support

    negative       0.78      0.61      0.68       534
    positive       0.96      0.98      0.97      4702

    accuracy                           0.94      5236
   macro avg       0.87      0.79      0.83      5236
weighted avg       0.94      0.94      0.94      5236

